In [4]:
import os
import warnings
warnings.filterwarnings('ignore')

import numpy as np

from glob import glob
from minisom import MiniSom
from pathlib import Path

INTERVAL = 15

In [5]:
def dropna(nparray):
    if isinstance(nparray[0], np.ndarray):
        return np.array([dropna(x) for x in nparray])
    else:
        return nparray[~np.isnan(nparray)]

In [6]:
for direction in ["src", "dst"]:
    for dirname in glob(Path(f'timeseries/pch/20230329/interval_{INTERVAL}_{direction}*').__str__()):
        print("Dir: ", dirname)
        # if os.path.exists(os.path.join(dirname, "ksom_label.npy")):
        #     continue
        pyts_dataset = np.load(os.path.join(dirname, "pyts_dataset.npy"))
        pyts_dataset = dropna(pyts_dataset)
        print("\tPyts dataset shape: ", pyts_dataset.shape)
        
        
        X = pyts_dataset.reshape(len(pyts_dataset), -1)
        print("\tX shape: ", X.shape)
        
        som_x = som_y = 10
        som = MiniSom(som_x, som_y, len(X[0]), sigma=0.3, learning_rate = 0.1, random_seed=10)
        som.random_weights_init(X)
        som.train(X, 50000, verbose=True)
        y_pred = np.array([som.winner(X[idx])[1] for idx in range(len(X))])
        
        np.save(os.path.join(dirname, "ksom_label.npy"), y_pred)
        print("\tKernel SOM shape: ", y_pred.shape)
        del pyts_dataset, som, X, y_pred

Dir:  timeseries\pch\20230330\interval_15_src_feature
	Pyts dataset shape:  (1563, 8, 61)
	X shape:  (1563, 488)
 [ 50000 / 50000 ] 100% - 0:00:00 left 
 quantization error: 0.0
	Kernel SOM shape:  (1563,)
Dir:  timeseries\pch\20230330\interval_15_dst_feature
	Pyts dataset shape:  (4731, 8, 61)
	X shape:  (4731, 488)
 [ 50000 / 50000 ] 100% - 0:00:00 left 
 quantization error: 0.0
	Kernel SOM shape:  (4731,)
